In [1]:
import pandas as pd
import numpy as np
import math
import random 
import time
import string
import glob

def clean_text(text):
    # Handle common misinterpretations from double encoding
    replacements = {
        'Ã¡': 'á', 'Ã©': 'é', 'Ã\xad': 'í', 'Ã³': 'ó', 'Ãº': 'ú',
        'Ã£': 'ã', 'Ãµ': 'õ', 'Ã¢': 'â', 'Ãª': 'ê', 'Ã´': 'ô',
        'Ã§': 'ç', 'Ã ': 'à', 'Ãš': 'Ú', 'Ã\x81': 'Á', 'Ã‰': 'É',
        'Ã\x8d': 'Í', 'Ã“': 'Ó', 'Ãš': 'Ú', 'Ã“': 'Ó'
    }
    for wrong, right in replacements.items():
        text = text.replace(wrong, right)
    return text

def read_files(file, method):
    with open(file, 'r', encoding="latin1") as f:
        lines = f.readlines()
        for l in lines:
            if l.strip():  # This checks that the line is not empty or just a newline
                # Decode and correct the encoding issues before evaluation
                corrected_line = l.encode('latin1').decode('utf-8', errors='replace')
                corrected_line = clean_text(corrected_line)
                
                # Use eval to convert string to list of tuples
                data = list(eval(corrected_line))
                
                data.sort(key=lambda tup: tup[1], reverse=True)
                pred[method].append([i[0].replace(' ', '') for i in data if i[1] > 0])
                
pred = {}

for method in ['lime', 'shap']:
    pred[method] = []
    paths = []
    for name in glob.glob('../results/ptt5_'+method+'*'):
        paths.append(name)

    for filename in sorted(paths):    
        read_files(filename, method)

In [2]:
df = pd.read_csv('../data/hatebr_and_rationales.csv', \
                index_col=0, \
                converters={"rationales_offensive_1_normalized": \
                lambda x: x.strip('[]').replace("'", "").split(", "),
                "rationales_offensive_2_normalized": \
                lambda x: x.strip('[]').replace("'", "").split(", ")})


In [3]:
## split to train and val
TRAIN_SIZE = 0.8
TEST_SIZE = 0.1
VAL_SIZE = 0.1
from sklearn.model_selection import train_test_split


x_train, x_test_val, y_train, y_test_val = train_test_split(df['normalized_text'], df['label final'], test_size=TEST_SIZE + VAL_SIZE, random_state=0)
x_test, x_val, y_test, y_val = train_test_split(x_test_val, y_test_val, test_size=VAL_SIZE/(TEST_SIZE + VAL_SIZE), random_state=0)


In [4]:
def get_x_rationales(method):
    rationales = []
    for e, p in enumerate(pred[method]):
        rationales.append(' '.join(p))
    
    return rationales

def get_x():
    df_instances = x_train.iloc[instances].to_frame().join(df.set_index('normalized_text'), on='normalized_text')
    X = df_instances['normalized_text'].tolist()
    return X

def remove_rationales(method):
    X = get_x()
    w_punct_intances = []
    for i in X:
        w_punct_intances.append(i.translate(str.maketrans('', '', string.punctuation)))

    n = math.ceil((np.mean([len(a) for a in pred[method]])))
    new_X = []

    for a, b in zip(pred[method], w_punct_intances):
        
        x = b.split(' ')

        save = []
        i = 0
        while i < n and i < len(a):
            try:
                x.remove(a[i])
                i+=1
            except:
                save.append(a[i])
                i+=1

        b = ' '.join(x)

        for a in save:
            b = b.replace(a, '')
        
        new_X.append(b)

    return new_X 

### Instanciando Modelo PTT5 pre-treinado

In [5]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForSequenceClassification,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

from torch.utils.data import TensorDataset
import torch
import numpy as np
from transformers import T5Tokenizer

def tokenize_corpus(df, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for doc in df:
        # Tokenizing input text
        encoded_dict = tokenizer.encode_plus(
                            doc,
                            add_special_tokens=True,
                            max_length=max_len,
                            truncation=True,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt'
                       )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

def prepare_dataset(features, labels, tokenizer):
    # Tokenize the input texts
    padded_tokens, attention_masks = tokenize_corpus(features.values, tokenizer, 512) # tokeniza as mensagens
    target = np.array(labels.values, dtype=np.int64).reshape(-1, 1) # transforma target em np array
    tensor_df = TensorDataset(padded_tokens, attention_masks, torch.from_numpy(target))

    return tensor_df


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from transformers import T5ForSequenceClassification, T5Tokenizer

# Specify the path or model identifier
model_name_or_path = 'unicamp-dl/ptt5-base-portuguese-vocab'  # Or use a model identifier like 't5-small'

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(model_name_or_path)

# Load the model
model = T5ForSequenceClassification.from_pretrained(model_name_or_path)

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at unicamp-dl/ptt5-base-portuguese-vocab and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Load the checkpoint
checkpoint = torch.load('../models/ptt5-fine-tuned/best-checkpoint.ckpt') #, map_location=torch.device('cpu'))

# Extract the state_dict from the checkpoint
state_dict = checkpoint['state_dict']

# Create a new dictionary that matches the expected keys
new_state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}

# Load the state dictionary into the model
model.load_state_dict(new_state_dict)

/tmp/ipykernel_23/3256266520.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/kaggle/input/ptt5-fine-tuned/best-checkpoint.ckpt') #, map_locatio

<All keys matched successfully>

In [8]:
model.cuda()

T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feat

In [9]:
import torch.nn.functional as F

def predict_fn(text):
    model.eval()
    
    padded_tokens, attention_masks = tokenize_corpus(text, tokenizer, 512) # tokeniza as mensagens
    tensor_df = TensorDataset(padded_tokens, attention_masks)
    
    test_dataloader = DataLoader(tensor_df,
                              batch_size=8,
                              shuffle=False)
        
    probabilities = []
    
    for batch in test_dataloader:
        
        with torch.no_grad():
            outputs = model(input_ids=batch[0].cuda(),attention_mask= batch[1].cuda())
    
        # Get the logits (raw predictions)
        logits = outputs.logits

        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)
        torch.cuda.empty_cache()
        probabilities.extend(probs.cpu().numpy())
        
    # Return probabilities as a numpy array (LIME and SHAP expect numpy arrays)
    return probabilities

In [10]:
X = get_x()

In [11]:
pred_X = predict_fn(X)

In [12]:
for method in ['lime', 'shap']:
    X_ = remove_rationales(method)
    pred_X_ = predict_fn(X_)

    x_rationales = get_x_rationales(method)
    pred_x_rationales = predict_fn(x_rationales)


    comprehensiveness = []
    for p1, p2 in zip(pred_X, pred_X_):
        comprehensiveness.append(p1[1] - p2[1])

   
    print("Comprehensiveness method {} value {}".format(method, np.mean(comprehensiveness)))

    sufficiency = []

    for p1, p2 in zip(pred_X, pred_x_rationales):
        sufficiency.append(p1[1] - p2[1])

    print("Sufficiency method {} value {}".format(method, np.mean(sufficiency)))


Comprehensiveness method lime value 0.56540447473526
Sufficiency method lime value 0.0016100808279588819
Comprehensiveness method shap value 0.6159740090370178
Sufficiency method shap value 0.008297599852085114


Get probabilities for test dataset to compute ROC curve

In [13]:
df_instances = x_test.to_frame().join(df.set_index('normalized_text'), on='normalized_text')
X = df_instances['normalized_text'].tolist()

logits = predict_fn(X)
probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()

y_pred = probs[:, 1] 

np.savetxt('../predictions/ptt5_pred.out', y_pred, delimiter=',') 

/tmp/ipykernel_23/3955221534.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  probs = torch.softmax(torch.tensor(logits), dim=-1).numpy()
